In [2]:
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Flatten, LSTM, Bidirectional
from keras.models import Sequential
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 選擇數據處理模式：'N-R' 或 'R'
mode = 'R'  # 可以選擇 'N-R' 或 'R'

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2014test-A.txt',
    './dataset/train/twitter-2015train-A.txt'
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # Non-Regional 模式，直接將推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # Regional 模式，將推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練 Word2Vec 模型
tokenized_sentences = [tweet.split() for tweet in dataset['tweet'].values]
word2vec_model = Word2Vec(sentences=tokenized_sentences,
                          vector_size=EMBEDDING_DIM,  # 詞向量維度為 25
                          window=5,                   # 最大跳躍距離設為 5
                          min_count=5,                # 最低頻次為 5
                          sg=0,                       # CBOW 模型 (sg=0)
                          workers=4)                  # 使用 4 個核心進行訓練

# 將詞向量進行標準化
def normalize_vector(vector):
    v_min, v_max = np.min(vector), np.max(vector)
    return (vector - v_min) / (v_max - v_min) if v_max > v_min else vector

# 構建嵌入矩陣
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_vector = word2vec_model.wv[word]
        embedding_matrix[i] = normalize_vector(embedding_vector)

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

In [3]:
from keras.layers import Flatten

# 建立單一 CNN 模型並加入 Flatten 層
cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 1s 3ms/step - loss: 1.0044 - accuracy: 0.4588 - val_loss: 0.9776 - val_accuracy: 0.5037
Epoch 2/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9811 - accuracy: 0.4878 - val_loss: 0.9688 - val_accuracy: 0.5046
Epoch 3/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9721 - accuracy: 0.5012 - val_loss: 0.9703 - val_accuracy: 0.4888
Epoch 4/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9677 - accuracy: 0.5045 - val_loss: 0.9684 - val_accuracy: 0.5237
Epoch 5/10
301/301 [==============================] - 1s 3ms/step - loss: 0.9653 - accuracy: 0.5061 - val_loss: 0.9663 - val_accuracy: 0.5179
Epoch 6/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9637 - accuracy: 0.5050 - val_loss: 0.9663 - val_accuracy: 0.5175
Epoch 7/10
301/301 [==============================] - 1s 2ms/step - loss: 0.9609 - accuracy: 0.5062 - val_loss: 0.9613 - val_accuracy: 0.5212
Epoch 

In [4]:
# 單一 LSTM 模型 (Single LSTM)

from keras.layers import LSTM

# 建立單一 LSTM 模型
lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 28s 86ms/step - loss: 1.0090 - accuracy: 0.4497 - val_loss: 0.9936 - val_accuracy: 0.4759
Epoch 2/10
301/301 [==============================] - 25s 84ms/step - loss: 0.9946 - accuracy: 0.4692 - val_loss: 1.0034 - val_accuracy: 0.3828
Epoch 3/10
301/301 [==============================] - 25s 82ms/step - loss: 0.9839 - accuracy: 0.4851 - val_loss: 0.9749 - val_accuracy: 0.5017
Epoch 4/10
301/301 [==============================] - 24s 80ms/step - loss: 0.9774 - accuracy: 0.4880 - val_loss: 0.9769 - val_accuracy: 0.4850
Epoch 5/10
301/301 [==============================] - 25s 83ms/step - loss: 0.9782 - accuracy: 0.4940 - val_loss: 0.9678 - val_accuracy: 0.4950
Epoch 6/10
301/301 [==============================] - 25s 84ms/step - loss: 0.9740 - accuracy: 0.4953 - val_loss: 0.9708 - val_accuracy: 0.5091
Epoch 7/10
301/301 [==============================] - 26s 87ms/step - loss: 0.9765 - accuracy: 0.4897 - val_loss: 0.9592 - val_accuracy:

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
# CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

# 建立 CNN + LSTM 模型
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 41s 123ms/step - loss: 1.0009 - accuracy: 0.4631 - val_loss: 0.9766 - val_accuracy: 0.4958
Epoch 2/10
301/301 [==============================] - 6s 21ms/step - loss: 0.9790 - accuracy: 0.4870 - val_loss: 0.9752 - val_accuracy: 0.4958
Epoch 3/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9760 - accuracy: 0.4930 - val_loss: 0.9544 - val_accuracy: 0.5067
Epoch 4/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9732 - accuracy: 0.4939 - val_loss: 0.9571 - val_accuracy: 0.5058
Epoch 5/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9717 - accuracy: 0.5011 - val_loss: 0.9513 - val_accuracy: 0.5112
Epoch 6/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9679 - accuracy: 0.5001 - val_loss: 0.9544 - val_accuracy: 0.5166
Epoch 7/10
301/301 [==============================] - 6s 20ms/step - loss: 0.9679 - accuracy: 0.4997 - val_loss: 0.9599 - val_accuracy: 0.50

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

76/76 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0       0.46      0.49      0.47       921
           1       0.00      0.00      0.00       341
           2       0.56      0.70      0.62      1144

    accuracy                           0.52      2406
   macro avg       0.34      0.39      0.36      2406
weighted avg       0.44      0.52      0.47      2406



c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
# 單一 3 層 CNN 和 LSTM 網絡

three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 6s 12ms/step - loss: 1.0006 - accuracy: 0.4611 - val_loss: 0.9677 - val_accuracy: 0.5104
Epoch 2/10
301/301 [==============================] - 3s 11ms/step - loss: 0.9817 - accuracy: 0.4857 - val_loss: 0.9743 - val_accuracy: 0.4896
Epoch 3/10
301/301 [==============================] - 3s 11ms/step - loss: 0.9729 - accuracy: 0.4931 - val_loss: 0.9582 - val_accuracy: 0.5116
Epoch 4/10
301/301 [==============================] - 3s 11ms/step - loss: 0.9639 - accuracy: 0.5025 - val_loss: 0.9685 - val_accuracy: 0.4979
Epoch 5/10
301/301 [==============================] - 3s 10ms/step - loss: 0.9616 - accuracy: 0.5035 - val_loss: 0.9546 - val_accuracy: 0.5096
Epoch 6/10
301/301 [==============================] - 3s 10ms/step - loss: 0.9574 - accuracy: 0.5031 - val_loss: 1.0042 - val_accuracy: 0.4609
Epoch 7/10
301/301 [==============================] - 3s 10ms/step - loss: 0.9496 - accuracy: 0.5162 - val_loss: 0.9552 - val_accuracy: 0.5083

In [8]:
# 多重 CNN 和 LSTM 網絡

multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 8s 20ms/step - loss: 0.9990 - accuracy: 0.4696 - val_loss: 0.9849 - val_accuracy: 0.4954
Epoch 2/10
301/301 [==============================] - 6s 18ms/step - loss: 0.9889 - accuracy: 0.4774 - val_loss: 0.9784 - val_accuracy: 0.5100
Epoch 3/10
301/301 [==============================] - 6s 19ms/step - loss: 0.9743 - accuracy: 0.4945 - val_loss: 0.9606 - val_accuracy: 0.4988
Epoch 4/10
301/301 [==============================] - 6s 18ms/step - loss: 0.9711 - accuracy: 0.5041 - val_loss: 0.9733 - val_accuracy: 0.5012
Epoch 5/10
301/301 [==============================] - 6s 18ms/step - loss: 0.9709 - accuracy: 0.5014 - val_loss: 0.9557 - val_accuracy: 0.5150
Epoch 6/10
301/301 [==============================] - 6s 19ms/step - loss: 0.9643 - accuracy: 0.5077 - val_loss: 0.9496 - val_accuracy: 0.5104
Epoch 7/10
301/301 [==============================] - 6s 18ms/step - loss: 0.9595 - accuracy: 0.5103 - val_loss: 0.9503 - val_accuracy: 0.5154

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
from keras.layers import Bidirectional

# 單一 3 層 CNN 和雙向 LSTM 網絡

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 10s 18ms/step - loss: 0.9963 - accuracy: 0.4748 - val_loss: 0.9695 - val_accuracy: 0.4888
Epoch 2/10
301/301 [==============================] - 5s 17ms/step - loss: 0.9762 - accuracy: 0.4963 - val_loss: 0.9657 - val_accuracy: 0.5179
Epoch 3/10
301/301 [==============================] - 5s 17ms/step - loss: 0.9693 - accuracy: 0.5052 - val_loss: 0.9654 - val_accuracy: 0.5150
Epoch 4/10
301/301 [==============================] - 5s 17ms/step - loss: 0.9647 - accuracy: 0.5060 - val_loss: 0.9523 - val_accuracy: 0.5187
Epoch 5/10
301/301 [==============================] - 5s 17ms/step - loss: 0.9579 - accuracy: 0.5078 - val_loss: 0.9496 - val_accuracy: 0.5154
Epoch 6/10
301/301 [==============================] - 5s 17ms/step - loss: 0.9562 - accuracy: 0.5119 - val_loss: 0.9498 - val_accuracy: 0.5258
Epoch 7/10
301/301 [==============================] - 5s 17ms/step - loss: 0.9491 - accuracy: 0.5165 - val_loss: 0.9469 - val_accuracy: 0.515

c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jk121\.conda\envs\DL-113\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
# 多重 CNN 和雙向 LSTM 網絡

multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
301/301 [==============================] - 15s 35ms/step - loss: 1.0038 - accuracy: 0.4542 - val_loss: 0.9771 - val_accuracy: 0.4759
Epoch 2/10
301/301 [==============================] - 11s 35ms/step - loss: 0.9771 - accuracy: 0.4966 - val_loss: 0.9677 - val_accuracy: 0.5042
Epoch 3/10
301/301 [==============================] - 11s 35ms/step - loss: 0.9726 - accuracy: 0.4975 - val_loss: 0.9564 - val_accuracy: 0.5104
Epoch 4/10
301/301 [==============================] - 11s 37ms/step - loss: 0.9635 - accuracy: 0.5061 - val_loss: 0.9617 - val_accuracy: 0.5087
Epoch 5/10
301/301 [==============================] - 11s 35ms/step - loss: 0.9621 - accuracy: 0.5080 - val_loss: 0.9785 - val_accuracy: 0.4859
Epoch 6/10
301/301 [==============================] - 11s 35ms/step - loss: 0.9611 - accuracy: 0.5077 - val_loss: 0.9536 - val_accuracy: 0.5079
Epoch 7/10
301/301 [==============================] - 11s 36ms/step - loss: 0.9571 - accuracy: 0.5085 - val_loss: 0.9579 - val_accuracy: